In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [2]:
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1]/72.0,frames[0].shape[0]/72.0),dpi=72.0)
    patch=plt.imshow(frames[0])
    plt.axis('off')
    
    def animate(i):
        patch.set_data(frames[i])
        
    anim=animation.FuncAnimation(plt.gcf(),animate,frames=len(frames),interval=50)
    anim.save('movie-cartpole-DQN.mp4')
    display(display_animation(anim,default_mode='loop'))
    

In [3]:
#名前をつけて値を格納していく
from collections import namedtuple
Transition=namedtuple('Transition',('state','action','next_state','reward'))

In [4]:
#経験を保存するクラスを定義
import random
class ReplayMemory():
    def __init__(self,capacity):
        self.capacity=capacity
        self.memory=[]
        self.index=0#保存するインデックスを示す変数
        
    def push(self,state,action,next_state,reward):
        if len(self.memory)<self.capacity:
            self.memory.append(None)
            
        self.memory[self.index]=Transition(state,action,next_state,reward)
        self.index=(self.index+1)%self.capacity
        
    def sample(self,batch_size):
        return random.sample(self.memory,batch_size)
    
    #memoryの長さを返す
    def __len__(self):
        return len(self.memory)

In [ ]:
#agentの脳みそ
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D
from keras.layers import Dropout,Flatten
from keras.optimizers import RMSprop


batch_size=32
capacity=10000

class Brain():
    def __init__(self,num_states,num_actions):
        #行動選択の個数
        self.num_actions=num_actions
        
        self.memory=ReplayMemory(capacity)
        
        #脳みその構造を構築 selfで脳構造をインスタンス化
        self.model=Sequential()
        self.model.add(Dense(32,activation='relu',input_shape=(num_states)))
        self.model.add(Dense(32,activation='relu'))
        self.model.add(Dense(num_actions,activation='relu'))
    
    def replay(self):
        if len(self.memory)<batch_size:
            return
        #遷移　メモリーの中から抽出
        transitions=self.memory.sample(batch_size)
        #(state,action,next_state,reward)*batch_size==>>
        #(state*batch_size,action*batch_size,next_state*batch_size,reward*batch_size)
        batch=Transitions(*zip(*transitions))
        
        state_batch=np.concatenate(batch.state)
        action_batch=np.concatenate(batch.action)
        reward_batch=np.concatenate(batch.reward)
        
        non_final_next_state=np.concatenate([s for s in batch.next_state if s is not None])
        
        
        